In [1]:
import pandas as pd
import numpy as np

# =====================================================
# 1️⃣ 데이터 로드
# =====================================================
csv_path = "new_flight_weather_merged.csv"
df = pd.read_csv(csv_path)

print("✅ CSV 로드 완료:", len(df))

C:\Users\Admin\AppData\Local\Temp\ipykernel_10200\2262982101.py:8: DtypeWarning: Columns (28,49) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


✅ CSV 로드 완료: 2843934


In [2]:
# =====================================================
# 2️⃣ 시간 파생 변수
# =====================================================
df["departure_datetime"] = pd.to_datetime(df["departure_datetime"])

df["dep_hour"] = df["departure_datetime"].dt.hour
df["dep_weekday"] = df["departure_datetime"].dt.weekday
df["is_weekend"] = df["dep_weekday"].isin([5, 6]).astype(int)

In [3]:
# =====================================================
# 3️⃣ 컬럼 선택
# =====================================================

num_cols = [
    "기온(°C)",
    "풍속_ms",
    "dep_hour",
    "dep_weekday",
    "is_weekend"
]
num_cols = [c for c in num_cols if c in df.columns]

cat_cols = [
    "항공사",
    "출발지",
    "도착지",
    "flight_type"
]
cat_cols = [c for c in cat_cols if c in df.columns]

X_cols = num_cols + cat_cols

In [4]:
# =====================================================
# 4️⃣ Train / Test 분리 (시간 기준)
# =====================================================
df = df.sort_values("departure_datetime")
split_date = df["departure_datetime"].quantile(0.8)

train_df = df[df["departure_datetime"] <= split_date]
test_df  = df[df["departure_datetime"] > split_date]

X_train = train_df[X_cols]
y_train = train_df["is_delay"]

X_test  = test_df[X_cols]
y_test  = test_df["is_delay"]

print("Train:", len(train_df), "Test:", len(test_df))

Train: 2275147 Test: 568787


In [5]:
# =====================================================
# 5️⃣ 전처리 (XGBoost용)
# =====================================================
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

num_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="median"))
])

cat_pipeline = Pipeline([
    ("imputer", SimpleImputer(strategy="constant", fill_value="UNKNOWN")),
    ("onehot", OneHotEncoder(
        handle_unknown="ignore",
        sparse_output=False
    ))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", num_pipeline, num_cols),
        ("cat", cat_pipeline, cat_cols)
    ]
)

In [6]:
import sys
!{sys.executable} -m pip install xgboost

In [7]:
# =====================================================
# 6️⃣ XGBoost 모델 정의
# =====================================================
from xgboost import XGBClassifier

xgb_model = XGBClassifier(
    n_estimators=400,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,

    # 🔥 불균형 대응 핵심
    scale_pos_weight = (y_train == 0).sum() / (y_train == 1).sum(),

    objective="binary:logistic",
    eval_metric="aucpr",
    tree_method="hist",
    random_state=42,
    n_jobs=-1
)

model = Pipeline([
    ("prep", preprocessor),
    ("xgb", xgb_model)
])

In [8]:
# =====================================================
# 7️⃣ 모델 학습
# =====================================================
model.fit(X_train, y_train)
print("✅ XGBoost 학습 완료")

✅ XGBoost 학습 완료


In [9]:
# =====================================================
# 8️⃣ 평가
# =====================================================
from sklearn.metrics import (
    classification_report,
    roc_auc_score,
    average_precision_score
)

y_prob = model.predict_proba(X_test)[:, 1]
y_pred = (y_prob >= 0.4).astype(int)  # 운영 기준

print("\n📊 기본 성능 (threshold=0.4)")
print(classification_report(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, y_prob))
print("PR-AUC :", average_precision_score(y_test, y_prob))


📊 기본 성능 (threshold=0.4)
              precision    recall  f1-score   support

           0       0.88      0.43      0.58    422243
           1       0.34      0.83      0.48    146544

    accuracy                           0.53    568787
   macro avg       0.61      0.63      0.53    568787
weighted avg       0.74      0.53      0.55    568787

ROC-AUC: 0.6840748932531735
PR-AUC : 0.424496566638183


In [10]:
# =====================================================
# 9️⃣ Threshold 실험
# =====================================================
for t in [0.25, 0.3, 0.35, 0.4, 0.45]:
    print(f"\n===== Threshold = {t} =====")
    y_pred_t = (y_prob >= t).astype(int)
    print(classification_report(y_test, y_pred_t))


===== Threshold = 0.25 =====
              precision    recall  f1-score   support

           0       0.93      0.18      0.30    422243
           1       0.29      0.96      0.44    146544

    accuracy                           0.38    568787
   macro avg       0.61      0.57      0.37    568787
weighted avg       0.76      0.38      0.33    568787


===== Threshold = 0.3 =====
              precision    recall  f1-score   support

           0       0.91      0.26      0.41    422243
           1       0.30      0.92      0.46    146544

    accuracy                           0.43    568787
   macro avg       0.61      0.59      0.43    568787
weighted avg       0.75      0.43      0.42    568787


===== Threshold = 0.35 =====
              precision    recall  f1-score   support

           0       0.90      0.34      0.50    422243
           1       0.32      0.89      0.47    146544

    accuracy                           0.48    568787
   macro avg       0.61      0.61      